In [1]:
import requests
import snowflake.connector
from datetime import datetime
import os

# Define your Mockaroo API key and schema name
api_key = '647272f0'  # Replace with your actual Mockaroo API key
schema_name = 'subscriber_highlevel'  # Replace with your actual schema name

# Snowflake connection parameters (update with your details)
conn = snowflake.connector.connect(
    connection_name = 'DEMO_MQUARFOT',
    warehouse = 'mitch_devwork',
    database = 'ad_data',
    schema = 'demo_data'
)

# Name of the stage and table in Snowflake
stage_name = 'MOCKAROO_STAGE'
table_name = 'MOCKAROO_DATA'


In [2]:
# Function to generate CSV data and upload to Snowflake stage
def generate_and_upload_csv():
    # Construct the URL to call the existing schema (with .csv extension)
    url = f'https://api.mockaroo.com/api/generate.csv?key={api_key}&schema={schema_name}'

    # Specify the number of records you want to generate
    params = {
        'count': 1000  # Adjust this number to the amount of data records you need
    }

    # Send the GET request to generate data from your saved schema
    response = requests.get(url, params=params)

    # Check if the request was successful and proceed
    if response.status_code == 200:
        # Create a filename with the current timestamp for the CSV
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        file_name = f'mockaroo_data_{timestamp}.csv'
        local_file_path = f'/tmp/{file_name}'

        # Write the content of the response (CSV data) to a temporary file
        with open(local_file_path, 'wb') as file:
            file.write(response.content)
        
        print(f'CSV file successfully created locally: {local_file_path}')

        # Upload the CSV file to a Snowflake stage
        upload_to_snowflake_stage(local_file_path, file_name)
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}, Message: {response.text}")

In [3]:
# Function to upload CSV to Snowflake stage
def upload_to_snowflake_stage(local_file_path, file_name):
    # Create a stage if it doesn't exist
    conn.cursor().execute(f'CREATE STAGE IF NOT EXISTS {stage_name}')
    print(f'Stage {stage_name} created or already exists.')

    # Upload the file to the stage
    conn.cursor().execute(f'PUT file://{local_file_path} @{stage_name}/{file_name}')
    print(f'File {file_name} uploaded to stage {stage_name}.')

    # Create a table if it doesn't exist and load the data
    load_data_to_snowflake_table(file_name)

In [4]:
# Function to create a Snowflake table and load data from the stage
def load_data_to_snowflake_table(file_name):
    # Define the SQL to create a table based on your schema (customize the columns as needed)
    create_table_sql = f'''
    CREATE OR REPLACE TABLE {table_name} (
        subscriber_id varchar,
        email varchar,
        ip_address varchar,
        home_address varchar,
        city varchar,
        state varchar,
        zipcode string,
        avg_monthly_hours float,
        subscription_start_date date,
        subscription_end_date date
    );
    '''
    conn.cursor().execute(create_table_sql)
    print(f'Table {table_name} created or replaced.')

    # Load data from the stage into the table
    copy_into_sql = f'''
    COPY INTO {table_name}
    FROM @{stage_name}/{file_name}
    FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 1);
    '''
    conn.cursor().execute(copy_into_sql)
    print(f'Data loaded into {table_name} from file {file_name}.')

    # Optionally, remove the file from the stage after loading
    conn.cursor().execute(f'REMOVE @{stage_name}/{file_name}')
    print(f'File {file_name} removed from stage {stage_name}.')

In [5]:
# Call the function to generate the CSV and upload it to Snowflake
generate_and_upload_csv()

CSV file successfully created locally: /tmp/mockaroo_data_20241014_125840.csv
Stage MOCKAROO_STAGE created or already exists.
File mockaroo_data_20241014_125840.csv uploaded to stage MOCKAROO_STAGE.
Table MOCKAROO_DATA created or replaced.
Data loaded into MOCKAROO_DATA from file mockaroo_data_20241014_125840.csv.
File mockaroo_data_20241014_125840.csv removed from stage MOCKAROO_STAGE.


In [6]:
# Close the Snowflake connection
conn.close()